# QAR数据分析系统演示

本notebook演示QAR数据分析系统的主要功能，包括数据生成、可视化、安全性分析和成本优化。

## 1. 环境设置和导入

In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# 添加src目录到路径
sys.path.append('../src')

# 导入自定义模块
from data_generation.qar_simulator import QARDataSimulator
from visualization.qar_visualizer import QARVisualizer
from models.safety_models import AnomalyDetector, RiskAssessmentModel
from models.optimization_models import FuelOptimizationModel
from utils.data_processing import QARDataProcessor

print("✅ 模块导入成功")

## 2. 生成QAR模拟数据

In [ ]:
# 创建QAR数据模拟器
simulator = QARDataSimulator(
    flight_duration_minutes=90,  # 90分钟飞行
    sampling_rate_hz=1.0         # 1Hz采样率
)

# 生成完整数据集
qar_data = simulator.generate_complete_dataset()

# 添加异常数据
qar_data_with_anomalies = simulator.add_anomalies(qar_data, anomaly_rate=0.03)

print(f"生成了 {len(qar_data_with_anomalies)} 条QAR数据记录")
print(f"数据列数: {len(qar_data_with_anomalies.columns)}")
print(f"飞行阶段分布:")
print(qar_data_with_anomalies['FLIGHT_PHASE'].value_counts())

## 3. 数据预览

In [ ]:
# 显示数据基本信息
print("数据基本统计:")
key_params = ['ALT_STD', 'IAS', 'ENG_N1_1', 'ENG_FUEL_FLOW_1', 'ATT_PITCH']
display(qar_data_with_anomalies[key_params].describe())

# 显示前几行数据
print("\n数据样本:")
display(qar_data_with_anomalies[key_params].head())

## 4. 基础可视化分析

In [ ]:
# 创建可视化器
visualizer = QARVisualizer()

# 绘制飞行阶段和高度剖面
fig = visualizer.plot_flight_phases(qar_data_with_anomalies)
fig.show()

In [ ]:
# 绘制起飞参数趋势
fig = visualizer.plot_takeoff_parameters(qar_data_with_anomalies)
fig.show()

In [ ]:
# 绘制参数分布
fig = visualizer.plot_parameter_distributions(qar_data_with_anomalies)
plt.show()

In [ ]:
# 绘制相关性热力图
fig = visualizer.plot_correlation_heatmap(qar_data_with_anomalies)
plt.show()

## 5. 安全性分析

In [ ]:
# 异常检测
anomaly_detector = AnomalyDetector(contamination=0.05)
anomaly_detector.fit(qar_data_with_anomalies)

# 预测异常
anomaly_labels, anomaly_scores = anomaly_detector.predict(qar_data_with_anomalies)

# 统计异常
n_anomalies = np.sum(anomaly_labels == -1)
anomaly_rate = n_anomalies / len(anomaly_labels)

print(f"检测到 {n_anomalies} 个异常点")
print(f"异常率: {anomaly_rate:.2%}")

# 可视化异常分数
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(anomaly_scores)
plt.title('异常分数时间序列')
plt.xlabel('时间点')
plt.ylabel('异常分数')

plt.subplot(1, 2, 2)
plt.hist(anomaly_scores, bins=50, alpha=0.7)
plt.title('异常分数分布')
plt.xlabel('异常分数')
plt.ylabel('频次')

plt.tight_layout()
plt.show()

In [ ]:
# 风险评估
risk_assessor = RiskAssessmentModel()
risk_df = risk_assessor.assess_safety_risks(qar_data_with_anomalies)
safety_report = risk_assessor.generate_safety_report(risk_df)

print("安全风险评估报告:")
print(f"总记录数: {safety_report['total_records']}")
print(f"高风险时段: {safety_report['high_risk_periods']}")
print("风险等级分布:")
for level, count in safety_report['risk_summary'].items():
    print(f"  {level}: {count}")

print("\n风险因素统计:")
for factor, count in safety_report['risk_factors'].items():
    print(f"  {factor}: {count}")

In [ ]:
# 可视化风险分布
plt.figure(figsize=(12, 8))

# 风险等级分布
plt.subplot(2, 2, 1)
risk_counts = risk_df['RISK_LEVEL'].value_counts()
plt.pie(risk_counts.values, labels=risk_counts.index, autopct='%1.1f%%')
plt.title('风险等级分布')

# 风险分数时间序列
plt.subplot(2, 2, 2)
plt.plot(risk_df['TOTAL_RISK_SCORE'])
plt.title('风险分数时间序列')
plt.xlabel('时间点')
plt.ylabel('风险分数')

# 各类风险因素
plt.subplot(2, 2, 3)
risk_factors = ['ALTITUDE_RISK', 'SPEED_RISK', 'ENGINE_TEMP_RISK', 
                'FUEL_FLOW_RISK', 'PITCH_RISK', 'ROLL_RISK']
risk_counts = [risk_df[factor].sum() for factor in risk_factors]
plt.bar(range(len(risk_factors)), risk_counts)
plt.xticks(range(len(risk_factors)), [f.replace('_RISK', '') for f in risk_factors], rotation=45)
plt.title('各类风险因素统计')
plt.ylabel('风险次数')

# 高风险时段
plt.subplot(2, 2, 4)
high_risk_mask = risk_df['RISK_LEVEL'].isin(['HIGH', 'CRITICAL'])
plt.scatter(range(len(risk_df)), risk_df['ALT_STD'], 
           c=high_risk_mask, cmap='RdYlBu_r', alpha=0.6)
plt.title('高风险时段与高度关系')
plt.xlabel('时间点')
plt.ylabel('高度 (ft)')

plt.tight_layout()
plt.show()

## 6. 燃油优化分析

In [ ]:
# 燃油优化模型
fuel_optimizer = FuelOptimizationModel()
fuel_optimizer.fit(qar_data_with_anomalies)

# 预测燃油消耗
predicted_fuel = fuel_optimizer.predict_fuel_consumption(qar_data_with_anomalies)
actual_fuel = qar_data_with_anomalies['ENG_FUEL_FLOW_1'].values

# 计算节省潜力
fuel_savings = np.mean(actual_fuel) - np.mean(predicted_fuel)

print(f"平均实际燃油流量: {np.mean(actual_fuel):.2f} kg/h")
print(f"平均预测最优燃油流量: {np.mean(predicted_fuel):.2f} kg/h")
print(f"潜在燃油节省: {fuel_savings:.2f} kg/h ({fuel_savings/np.mean(actual_fuel)*100:.1f}%)")

In [ ]:
# 飞行剖面优化分析
optimization_results = fuel_optimizer.optimize_flight_profile(qar_data_with_anomalies)

if 'efficiency_analysis' in optimization_results:
    print("各飞行阶段燃油效率分析:")
    for phase, metrics in optimization_results['efficiency_analysis'].items():
        print(f"\n{phase}:")
        print(f"  平均燃油流量: {metrics['avg_fuel_flow']:.2f} kg/h")
        print(f"  平均速度: {metrics['avg_speed']:.2f} kt")
        print(f"  平均高度: {metrics['avg_altitude']:.0f} ft")
        print(f"  燃油效率: {metrics['fuel_efficiency']:.4f} kt/(kg/h)")
    
    print(f"\n最高效率阶段: {optimization_results['best_efficiency_phase']}")
    print(f"最低效率阶段: {optimization_results['worst_efficiency_phase']}")
    
    if 'recommendations' in optimization_results:
        print("\n优化建议:")
        for rec in optimization_results['recommendations']:
            print(f"  • {rec}")

In [ ]:
# 可视化燃油分析
plt.figure(figsize=(15, 10))

# 实际vs预测燃油消耗
plt.subplot(2, 3, 1)
plt.plot(actual_fuel, label='实际燃油流量', alpha=0.7)
plt.plot(predicted_fuel, label='预测最优燃油流量', alpha=0.7)
plt.title('燃油消耗对比')
plt.xlabel('时间点')
plt.ylabel('燃油流量 (kg/h)')
plt.legend()

# 燃油节省潜力
plt.subplot(2, 3, 2)
fuel_diff = actual_fuel - predicted_fuel
plt.plot(fuel_diff)
plt.title('燃油节省潜力')
plt.xlabel('时间点')
plt.ylabel('节省量 (kg/h)')
plt.axhline(y=0, color='r', linestyle='--', alpha=0.5)

# 燃油效率vs高度
plt.subplot(2, 3, 3)
fuel_efficiency = qar_data_with_anomalies['TAS'] / (qar_data_with_anomalies['ENG_FUEL_FLOW_1'] + 1e-6)
plt.scatter(qar_data_with_anomalies['ALT_STD'], fuel_efficiency, alpha=0.5)
plt.title('燃油效率 vs 高度')
plt.xlabel('高度 (ft)')
plt.ylabel('燃油效率 (kt/(kg/h))')

# 各阶段燃油消耗
plt.subplot(2, 3, 4)
phase_fuel = qar_data_with_anomalies.groupby('FLIGHT_PHASE')['ENG_FUEL_FLOW_1'].mean()
plt.bar(range(len(phase_fuel)), phase_fuel.values)
plt.xticks(range(len(phase_fuel)), phase_fuel.index, rotation=45)
plt.title('各阶段平均燃油消耗')
plt.ylabel('燃油流量 (kg/h)')

# 燃油流量分布
plt.subplot(2, 3, 5)
plt.hist(actual_fuel, bins=30, alpha=0.7, label='实际')
plt.hist(predicted_fuel, bins=30, alpha=0.7, label='预测最优')
plt.title('燃油流量分布')
plt.xlabel('燃油流量 (kg/h)')
plt.ylabel('频次')
plt.legend()

# 累积燃油消耗
plt.subplot(2, 3, 6)
cumulative_actual = np.cumsum(actual_fuel) / 3600  # 转换为kg
cumulative_predicted = np.cumsum(predicted_fuel) / 3600
plt.plot(cumulative_actual, label='实际累积消耗')
plt.plot(cumulative_predicted, label='预测最优累积消耗')
plt.title('累积燃油消耗')
plt.xlabel('时间点')
plt.ylabel('累积燃油消耗 (kg)')
plt.legend()

plt.tight_layout()
plt.show()

# 计算总节省
total_savings = (cumulative_actual[-1] - cumulative_predicted[-1])
print(f"\n本次飞行总燃油节省潜力: {total_savings:.2f} kg")
print(f"节省比例: {total_savings/cumulative_actual[-1]*100:.1f}%")

## 7. 总结报告

In [ ]:
print("="*60)
print("QAR数据分析系统演示总结报告")
print("="*60)

print(f"\n📊 数据概况:")
print(f"  • 飞行时长: {len(qar_data_with_anomalies)/60:.1f} 分钟")
print(f"  • 数据点数: {len(qar_data_with_anomalies):,} 条")
print(f"  • 参数数量: {len(qar_data_with_anomalies.columns)} 个")
print(f"  • 最大高度: {qar_data_with_anomalies['ALT_STD'].max():,.0f} ft")
print(f"  • 最大速度: {qar_data_with_anomalies['IAS'].max():.0f} kt")

print(f"\n🔍 安全性分析:")
print(f"  • 异常检测: 发现 {n_anomalies} 个异常点 ({anomaly_rate:.1%})")
print(f"  • 高风险时段: {safety_report['high_risk_periods']} 个")
print(f"  • 主要风险因素: {max(safety_report['risk_factors'], key=safety_report['risk_factors'].get)}")

print(f"\n⛽ 燃油优化:")
print(f"  • 平均燃油流量: {np.mean(actual_fuel):.0f} kg/h")
print(f"  • 优化后预测: {np.mean(predicted_fuel):.0f} kg/h")
print(f"  • 节省潜力: {fuel_savings:.0f} kg/h ({fuel_savings/np.mean(actual_fuel)*100:.1f}%)")
print(f"  • 本次飞行可节省: {total_savings:.1f} kg")

print(f"\n✅ 系统功能验证:")
print(f"  • ✓ QAR数据模拟生成")
print(f"  • ✓ 多维度可视化分析")
print(f"  • ✓ 异常检测与风险评估")
print(f"  • ✓ 燃油优化建议")
print(f"  • ✓ 机器学习模型集成")

print("\n" + "="*60)
print("演示完成！系统各项功能运行正常。")
print("="*60)